## Prototype of CNN model used in this paper: https://www.sciencedirect.com/science/article/pii/S2352914818301333

In [1]:
## Set environment variable for data reading
import os
os.environ["PTB_FOLDER_PATH"] = "/Users/cesargenatios/Downloads/ptb-diagnostic-ecg-database-1.0.0"

In [2]:
## Load data from PTB dataset
import sys
sys.path.insert(0,"/Users/cesargenatios/Desktop/cappy/ptd_data_handler")
from data_formatter import *
all_patient_data = get_formatted_ptb_data()


In [3]:
## Choose two patients one with MI, one with Non-MI to test PoC. Test on another 
mi_train_df = all_patient_data['patient024']['s0094lre']
healthy_train_df = all_patient_data['patient252']['s0487_re']

mi_test_df = all_patient_data['patient024']['s0083lre']

In [4]:
del all_patient_data

In [23]:
import numpy as np
## Grab 512 of one patient data
mi_train_data = mi_train_df[['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', 'v2', 'v3', 'v4', 'v5',
       'v6']].to_numpy()
mi_train_data = np.reshape(mi_train_data, [225,512,12])
mi_train_label = np.ones((225,1))
print(mi_train_data.shape, mi_train_label.shape) 

mi_test_data = mi_test_df[['i', 'ii', 'iii', 'avr', 'avl', 'avf', 'v1', 'v2', 'v3', 'v4', 'v5',
       'v6']].to_numpy()
mi_test_data = np.reshape(mi_test_data, [225,512,12])
mi_test_label = np.ones((225,1))
print(mi_test_data.shape, mi_test_label.shape)

(225, 512, 12) (225, 1)
(225, 512, 12) (225, 1)


## Model Architecture

In [17]:
import tensorflow as tf

beat_length = 512
num_feats = 12

## Layer 0 - input
input = tf.keras.Input(shape=(beat_length,num_feats))

## Layers 1-6 - convolutional block
x = tf.keras.layers.Conv1D(32, 3,  padding='same', activation='relu')(input)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool1D(pool_size=2)(x)
x = tf.keras.layers.Dropout(0.5)(x)

## Layers 7-12 - convolutional block
x = tf.keras.layers.Conv1D(32, 3,  padding='same', activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool1D(pool_size=2)(x)
x = tf.keras.layers.Dropout(0.5)(x)

## Layers 13-19 - convolutional block
x = tf.keras.layers.Conv1D(32, 3,  padding='same', activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool1D(pool_size=2)(x)
x = tf.keras.layers.Dropout(0.5)(x)

## Layers 19-24 - convolutional block
x = tf.keras.layers.Conv1D(32, 3,  padding='same', activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv1D(32, 3, padding='same', activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool1D(pool_size=2)(x)
x = tf.keras.layers.Dropout(0.5)(x)

## Layer 25 - flatten
x = tf.keras.layers.Flatten()(x)

## Layer 26 - dense
x = tf.keras.layers.Dense(32, activation='relu')(x)

## Layer 27 - batch norm
x = tf.keras.layers.BatchNormalization()(x)

## Layer 28 - dropout
x = tf.keras.layers.Dropout(0.5)(x)

## Layer 29 - dense
x = tf.keras.layers.Dense(32, activation='relu')(x)

## Layer 30 - batch norm
x = tf.keras.layers.BatchNormalization()(x)

## Layer 31 - dropout
x = tf.keras.layers.Dropout(0.5)(x)

## Layer 32 - dense
x = tf.keras.layers.Dense(16, activation='relu')(x)

## Layer 33 - batch norm
x = tf.keras.layers.BatchNormalization()(x)

## Layer 34 - dropout
x = tf.keras.layers.Dropout(0.5)(x)

## Layer 35 - output
output = tf.keras.layers.Dense(1, activation='softmax')(x)

cnn_model = tf.keras.Model(input, output, name="cnn_model")
cnn_model.summary()

Model: "cnn_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 512, 12)]         0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 512, 32)           1184      
_________________________________________________________________
batch_normalization_13 (Batc (None, 512, 32)           128       
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 512, 32)           3104      
_________________________________________________________________
batch_normalization_14 (Batc (None, 512, 32)           128       
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 256, 32)           0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 256, 32)           0 

In [18]:
tf.keras.utils.plot_model(cnn_model, "cnn_model.png", show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [19]:
cnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.BinaryCrossentropy(), metrics=tf.keras.metrics.BinaryAccuracy())

In [25]:
tf.config.experimental_run_functions_eagerly(True)

cnn_model.fit(x=mi_train_data, y=mi_train_label, batch_size=32, epochs=2, verbose=True, validation_data=(mi_test_data, mi_test_label))

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.
Epoch 1/2


/anaconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:4212: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


8/8 [==============================] - 3s 400ms/step - loss: 0.8811 - binary_accuracy: 1.0000 - val_loss: 0.6978 - val_binary_accuracy: 1.0000
Epoch 2/2
8/8 [==============================] - 3s 329ms/step - loss: 0.8082 - binary_accuracy: 1.0000 - val_loss: 0.6862 - val_binary_accuracy: 1.0000


In [27]:
tf.saved_model.save(cnn_model, "/Users/cesargenatios/Desktop/cappy/model_dev/model_prototypes/cnn/saved_model")

INFO:tensorflow:Assets written to: /Users/cesargenatios/Desktop/cappy/model_dev/cnn/saved_model/assets
